In [1]:
# %%writefile Generate_Base_Data.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
import warnings

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher,tools.Sample_Tools

def cache_single_stock_qfq(codes,self_obj=None):
    import tools.Sample_Tools as smpl
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    import warnings
    
    for code in codes:
        try:
            stock_df = smpl.get_data([code], start='2000-01-01', end='2023-01-01').to_qfq().data
            stock_df = smpl.add_marketvalue_industry(stock_df)
            dt_index = stock_df.index.get_level_values(0)
        except:
            # warnings.warn('get "{}" error'.format(code))
            continue

        try:
            save_cache('{}_train_qfq'.format(code) ,stock_df.loc[(dt_index >= '2016') & (dt_index < '2021')], cache_type=CACHE_TYPE.STOCK)
            save_cache('{}_tail_qfq'.format(code), stock_df.loc[(dt_index >= '2021')], cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('1.save_cache "{}" error'.format(code))
            continue

        try:
            save_cache('{}_older_qfq'.format(code), stock_df.loc[dt_index < '2016'],cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('2.save_cache "{}" error'.format(code))
            continue
        # print('{} saved'.format(code))
    

def generate_single_stock_qfq(codes, worker=6):
    task = task_chunk_split(codes, worker)
    results = parallal_task(worker, cache_stock_qfq, task)
    
    

def cache_all_stock_qfq(codes,self_obj=None):
    import tools.Sample_Tools as smpl
    import warnings
    
    try:
        stock_df = smpl.get_data(codes, start='2010-01-01', end='2023-01-01').to_qfq().data
        stock_df = smpl.add_marketvalue_industry(stock_df)
        dt_index = stock_df.index.get_level_values(0)
    except:
        warnings.warn('get "{}" error'.format(codes[0]))
        
    return stock_df

    
def generate_all_stock_qfq(codes, worker=6):
    task = task_chunk_split(codes, worker)
    results = parallal_task(worker, cache_all_stock_qfq, task)
    results = pd.concat(results).sort_index()
    dt_index = results.index.get_level_values(0)
    # try:
    save_cache('all_train_qfq' ,results.loc[(dt_index >= '2016') & (dt_index < '2021')], cache_type=CACHE_TYPE.STOCK)
    save_cache('all_tail_qfq', results.loc[(dt_index >= '2021')], cache_type=CACHE_TYPE.STOCK)
    # except:
    #     warnings.warn('1.save_cache error')

    # try:
    save_cache('all_older_qfq', results.loc[dt_index < '2016'],cache_type=CACHE_TYPE.STOCK)
    # except:
    #     warnings.warn('2.save_cache error')



In [2]:
# codes = smpl.get_codes_from_blockname('沪深300', sse='all')
codes = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)
generate_all_stock_qfq(codes,worker=10)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:5332  pid:3432


  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
df_all = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()
df_all.info()
# 7081385 

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7081385 entries, (Timestamp('2010-01-04 00:00:00'), '000001') to (Timestamp('2022-12-30 00:00:00'), '605599')
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   open                    float32
 1   high                    float32
 2   low                     float32
 3   close                   float32
 4   volume                  int32  
 5   amount                  int64  
 6   adj                     float32
 7   shares                  float64
 8   lshares                 float64
 9   totalCapital            float64
 10  liquidity_totalCapital  float64
 11  industry                object 
dtypes: float32(5), float64(4), int32(1), int64(1), object(1)
memory usage: 513.5+ MB
